# 2110443 - Computer Vision (2022/2)

# Lab 9 - Modern Object Detection 
In this lab, we will learn how to use Convolutional Neural Network on object detection problem by using famouse object detection framework <a href="https://github.com/open-mmlab/mmdetection">MMDetection</a>. This notebook includes both coding and written questions. Please hand in this notebook file with all outputs and your answer

Install prerequisite libraries for MMDetection
![mmdetection](https://raw.githubusercontent.com/open-mmlab/mmdetection/master/resources/mmdet-logo.png)

In [ ]:
!pip install openmim
!mim install mmdet

## MMDetection pretrained model (MaskRCNN)

In [ ]:
import mmcv
import numpy as np

In [ ]:
!mim download mmdet --config mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco --dest .


In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Read MaskRCNN config file
config = 'mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py'

# MaskRCNN pretrained on COCO dataset
checkpoint = 'mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# initialize the detector
maskRCNNModel = init_detector(config, checkpoint, device='cuda')

In [ ]:
# Read inputImage and apply detector
!wget https://raw.githubusercontent.com/open-mmlab/mmdetection/master/demo/demo.jpg
inputImage = 'demo.jpg'
result = inference_detector(maskRCNNModel, inputImage)

# Show Result
show_result_pyplot(maskRCNNModel, inputImage, result, score_thr=0.3)

# Raccoon Dataset
![Raccoon Dataset](https://i.imgur.com/cRQJ1PB.png)

Dataset URL : https://github.com/datitran/raccoon_dataset <br>
This dataset contains 196 images of raccoons and 213 bounding boxes (some images contain two raccoons). This is a single class problem, and images various size and scene condition. It's a great first dataset for getting started with object detection.



Download and extract preprocessed dataset from lab server

In [ ]:
!wget http://piclab.ai/classes/cv2020/raccoonsDataset.zip
!unzip -q raccoonsDataset.zip

### Dataset Exploration
We will use pycocotools to explore this dataset. 

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
trainLabelFile='raccoons/coco_annotations.json'
# initialize COCO api for instance annotations
trainCOCOBinding = COCO(trainLabelFile)

In [ ]:
#display COCO categories and supercategories
cats = trainCOCOBinding.loadCats(trainCOCOBinding.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

# get all images containing given categories, select one at random
catIds = trainCOCOBinding.getCatIds(catNms=['raccoon']);
imgIds = trainCOCOBinding.getImgIds(catIds=catIds );

randomImgId = np.random.randint(0,len(imgIds))
sampleImageData = trainCOCOBinding.loadImgs(imgIds[randomImgId])[0]

print('Image Data >>', sampleImageData)

sampleImage = cv2.imread('raccoons/'+sampleImageData['file_name'])

annIds = trainCOCOBinding.getAnnIds(imgIds=randomImgId, catIds=catIds, iscrowd=None)
boxes = trainCOCOBinding.loadAnns(annIds)
print('Box Data', boxes)

for box in boxes:
  x,y,w,h = box['bbox']
  cv2.rectangle(sampleImage, (int(x), int(y)), (int(x+w), int(y+h)), (0,255,0), 5)

sampleImage = cv2.cvtColor(sampleImage, cv2.COLOR_BGR2RGB)

plt.imshow(sampleImage)
plt.show()

### Modify MMDetection model configuration

In [ ]:
!mim download mmdet --config ssd300_coco --dest .


In [ ]:
from mmcv import Config


modelConfig = Config.fromfile('ssd300_coco.py') 


print(f'Original Config:\n{modelConfig.pretty_text}')


In [ ]:
from mmcv.utils.config import ConfigDict
from mmdet.apis import set_random_seed

# Modify dataset type and path
modelConfig.device = 'cuda'

modelConfig.dataset_type = 'CocoDataset'
modelConfig.data_root = './raccoons'
modelConfig.classes = ('raccoon',)

modelConfig.model.bbox_head.num_classes = 1

modelConfig.data.train = ConfigDict()
modelConfig.data.train.pipeline = modelConfig.train_pipeline
modelConfig.data.train.type = 'CocoDataset'
modelConfig.data.train.classes = ('raccoon',)
modelConfig.data.train.data_root = './raccoons'
modelConfig.data.train.ann_file = 'coco_annotations.json'
modelConfig.data.train.img_prefix = ''

modelConfig.data.val = ConfigDict()
modelConfig.data.val.pipeline = modelConfig.test_pipeline
modelConfig.data.val.type = 'CocoDataset'
modelConfig.data.val.classes =('raccoon',)
modelConfig.data.val.data_root = './raccoons'
modelConfig.data.val.ann_file = 'coco_annotations.json'
modelConfig.data.val.img_prefix = ''


modelConfig.data.test = ConfigDict()
modelConfig.data.test.pipeline = modelConfig.test_pipeline
modelConfig.data.test.type = 'CocoDataset'
modelConfig.data.test.classes =('raccoon',)
modelConfig.data.test.data_root = './raccoons'
modelConfig.data.test.ann_file = 'coco_annotations.json'
modelConfig.data.test.img_prefix = ''


# Set up working dir to save files and logs.
modelConfig.work_dir = './experiments'
# use pretrained model as start point
modelConfig.load_from = 'ssd300_coco_20210803_015428-d231a06e.pth'

modelConfig.optimizer.lr = 1e-3
modelConfig.lr_config.warmup = None
modelConfig.lr_config.policy = 'step'
modelConfig.lr_config.step = [10,25]
modelConfig.log_config.interval = 10

# Evaluation interval
modelConfig.evaluation.interval = 5
# Checkpoint saving interval
modelConfig.checkpoint_config.interval = 5
modelConfig.runner.max_epochs = 30

# Set seed thus the results are more reproducible
modelConfig.seed = 0
set_random_seed(0, deterministic=False)
modelConfig.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Modified Config:\n{modelConfig.pretty_text}')

### Training

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os

# Build dataset
datasets = [build_dataset(modelConfig.data.train)]

# Build the detector
model = build_detector(modelConfig.model, train_cfg=modelConfig.get('train_cfg'), test_cfg=modelConfig.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(modelConfig.work_dir))
train_detector(model, datasets, modelConfig, distributed=False, validate=True)

### Inference on image!

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
inputImage = mmcv.imread('raccoons/raccoon-115_jpg.rf.9723b0a68ad8ed8bdb5ccf6a210ba09b.jpg')

model.cfg = modelConfig
model.CLASSES = ('raccoons',)

result = inference_detector(model, inputImage)
show_result_pyplot(model, inputImage, result)

# Chula-RBC Dataset
![RBC Dataset](https://raw.githubusercontent.com/Chula-PIC-Lab/Chula-RBC-12-Dataset/main/Dataset/1.jpg)

Dataset URL : https://github.com/Chula-PIC-Lab/Chula-RBC-12-Dataset <br>

In [ ]:
!pip install gdown
!git clone https://github.com/Chula-PIC-Lab/Chula-RBC-12-Dataset
!gdown https://drive.google.com/uc?id=1EY0gPz5J4PVpFwiOUUFyjVqsl9yR4QyI

In [ ]:
!mim download mmdet --config vfnet_r50_fpn_1x_coco --dest .


### Dataset Exploration
We will use pycocotools to explore this dataset. 

In [ ]:
from mmcv import Config

modelConfig = Config.fromfile('vfnet_r50_fpn_1x_coco.py')
print(f'Original Config:\n{modelConfig.pretty_text}')


In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
trainLabelFile='rbcCocoGT.json'
# initialize COCO api for instance annotations
trainCOCOBinding = COCO(trainLabelFile)

In [ ]:
#display COCO categories and supercategories
cats = trainCOCOBinding.loadCats(trainCOCOBinding.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

# get all images containing given categories, select one at random
catIds = trainCOCOBinding.getCatIds(catNms=['Normal']);
imgIds = trainCOCOBinding.getImgIds(catIds=catIds );

randomImgId = np.random.randint(0,len(imgIds))
sampleImageData = trainCOCOBinding.loadImgs(randomImgId)[0]

print('Image Data >>', sampleImageData)

sampleImage = cv2.imread('Chula-RBC-12-Dataset/Dataset/'+sampleImageData['file_name'])

annIds = trainCOCOBinding.getAnnIds(imgIds=randomImgId, iscrowd=None)
boxes = trainCOCOBinding.loadAnns(annIds)
print('Box Data', boxes)

for box in boxes:
  x,y,w,h = box['bbox']
  cv2.rectangle(sampleImage, (int(x), int(y)), (int(x+w), int(y+h)), (0,255,0), 5)

sampleImage = cv2.cvtColor(sampleImage, cv2.COLOR_BGR2RGB)

plt.imshow(sampleImage)
plt.show()

### Modify MMDetection model configuration

In [ ]:
from mmcv.utils.config import ConfigDict
from mmdet.apis import set_random_seed

# Modify dataset type and path
modelConfig.device = 'cuda'

modelConfig.dataset_type = 'CocoDataset'
modelConfig.data_root = '/content/Chula-RBC-12-Dataset/Dataset'
modelConfig.classes = ('Normal', 'Macrocyte', 'Microcyte', 'Spherocyte', 'Target cell', 'Schistocyte', 'Hypochromia', 'Elliptocyte', 'Uncategorised')

modelConfig.data.train = ConfigDict()
modelConfig.data.train.pipeline = modelConfig.train_pipeline
modelConfig.data.train.type = 'CocoDataset'
modelConfig.data.train.classes = modelConfig.classes 
modelConfig.data.train.data_root = modelConfig.data_root
modelConfig.data.train.ann_file = '/content/rbcCocoGT.json'
modelConfig.data.train.img_prefix = ''

modelConfig.data.val = ConfigDict()
modelConfig.data.val.pipeline = modelConfig.test_pipeline
modelConfig.data.val.type = 'CocoDataset'
modelConfig.data.val.classes =modelConfig.classes 
modelConfig.data.val.data_root = modelConfig.data_root
modelConfig.data.val.ann_file = '/content/rbcCocoGT.json'
modelConfig.data.val.img_prefix = ''


modelConfig.data.test = ConfigDict()
modelConfig.data.test.pipeline = modelConfig.test_pipeline
modelConfig.data.test.type = 'CocoDataset'
modelConfig.data.test.classes =modelConfig.classes 
modelConfig.data.test.data_root = modelConfig.data_root
modelConfig.data.test.ann_file = '/content/rbcCocoGT.json'
modelConfig.data.test.img_prefix = ''


# Modify num classes of the model in box head
modelConfig.model.bbox_head.num_classes = 9

# use pretrained model as start point
modelConfig.load_from = 'vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth'

# Set up working dir to save files and logs.
modelConfig.work_dir = './RBCexperiments'

modelConfig.optimizer.lr = 1e-3
modelConfig.lr_config.warmup = None
modelConfig.lr_config.policy = 'step'
modelConfig.lr_config.step = [5,10]
modelConfig.log_config.interval = 10

# Evaluation interval
modelConfig.evaluation.interval = 5
# Checkpoint saving interval
modelConfig.checkpoint_config.interval = 5
modelConfig.runner.max_epochs = 5

# Set batch size
modelConfig.data.samples_per_gpu = 8

# Set seed thus the results are more reproducible
modelConfig.seed = 0
set_random_seed(0, deterministic=False)
modelConfig.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Modified Config:\n{modelConfig.pretty_text}')

### Training

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os

# Build dataset
datasets = [build_dataset(modelConfig.data.train)]

# Build the detector
model = build_detector(modelConfig.model, train_cfg=modelConfig.get('train_cfg'), test_cfg=modelConfig.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(modelConfig.work_dir))
train_detector(model, datasets, modelConfig, distributed=False, validate=True)

### Inference on image!

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
inputImage = mmcv.imread('/content/Chula-RBC-12-Dataset/Dataset/1.jpg')

model.cfg = modelConfig
model.CLASSES = ('Normal', 'Macrocyte', 'Microcyte', 'Spherocyte', 'Target cell', 'Schistocyte', 'Hypochromia', 'Elliptocyte', 'Uncategorised')

result = inference_detector(model, inputImage)
show_result_pyplot(model, inputImage, result)